In [109]:
import pandas as pd
import numpy as np
import sklearn
from scipy.stats import boxcox
from scipy.stats import percentileofscore

In [110]:
df = pd.read_csv('processed_data/df_3mth_modelled.csv').iloc[:, 1:]

# Qn 2: Creating Credit Score
Please create a credit score for each individual, validate your solution, and provide guidance on the next steps. 

In [111]:
def add_percentile_bins(df, column): 
    arr = df[column]
    df[str(column) + ' percentile'] = df[column].apply(lambda x: percentileofscore(arr, x))
    df[str(column) + ' percentile_round'] = df[str(column) + ' percentile'].apply(lambda x: round(x,0))
    
    bins = pd.IntervalIndex.from_tuples([(0, 10), (10, 20), (20, 30), (30,40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90,100)])
    df['perc_bins'] = pd.cut(df[str(column) + ' percentile'], bins, labels=["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"])
    df = df.reset_index()
    return df

In [112]:
df2 = add_percentile_bins(df, 'calibrated_prob')

In [113]:
# aggregated table indexed by default_flag
summary_df = pd.DataFrame(df2.groupby(['perc_bins', 'default_flag'])['default_flag'].count())
summary_df.columns = ['count']
summary_df = summary_df.reset_index()
summary_df['default_flag'] = summary_df['default_flag'].apply(lambda x: int(x))
summary_df = summary_df.pivot(index = 'perc_bins', values = 'count', columns = 'default_flag').reset_index()

# add total cases
summary_df['total_cases'] = summary_df[0] + summary_df[1]

# add cumulative counts for neg and pos cases
summary_df['cumulative_neg_case'] = summary_df[0].cumsum()
summary_df['cumulative_pos_case'] = summary_df[1].cumsum()

# add default rate and cumulative default rate
summary_df['default_rate'] = summary_df [1]/(summary_df [1] + summary_df [0])
summary_df['cumulative_default_rate'] = summary_df['cumulative_pos_case']/(summary_df['cumulative_neg_case'] + summary_df['cumulative_pos_case'])

summary_df = summary_df.reset_index()

In [137]:
summary_df = summary_df.rename(columns = {'perc_bins': 'credit_score'})

In [138]:
summary_df

default_flag,index,credit_score,0,1,total_cases,cumulative_neg_case,cumulative_pos_case,default_rate,cumulative_default_rate
0,0,"(0, 10]",1573,0,1573,1573,0,0.000000,0.000000
1,1,"(10, 20]",1569,0,1569,3142,0,0.000000,0.000000
2,2,"(20, 30]",1548,0,1548,4690,0,0.000000,0.000000
3,3,"(30, 40]",1561,7,1568,6251,7,0.004464,0.001119
4,4,"(40, 50]",1563,6,1569,7814,13,0.003824,0.001661
5,5,"(50, 60]",1551,10,1561,9365,23,0.006406,0.002450
6,6,"(60, 70]",1541,23,1564,10906,46,0.014706,0.004200
7,7,"(70, 80]",1529,35,1564,12435,81,0.022379,0.006472
8,8,"(80, 90]",1406,158,1564,13841,239,0.101023,0.016974
9,9,"(90, 100]",388,1177,1565,14229,1416,0.752077,0.090508


# Qn 1: Optimising Cut-off Rate
The upper management wants the overall default rate of their portfolio to be below 2.5%, please provide recommendations on the optimal credit score cutoff rate


In [ ]:
Optimal cut-off point to maintain <=2.5% default rate lies somewhere in the 90-100 credit score/percentile

In [116]:
def find_cutoff(df, credit_score, default_flag, target):
    default_rates = []

    for i in range(0,100):
       # print(i)

        if i == 0: 

            perc_group_0 = df[df[credit_score] == 0]
            try:
                default_rate = (len(perc_group_0[perc_group_0[default_flag] == 1]) /len(perc_group_0)) * 100
            except:
                continue

        if i == 1:
            perc_group = df[df[credit_score] == i]

            perc_group_added =  pd.concat([perc_group_0, perc_group])
            try:
                default_rate = (len(perc_group_added[perc_group_added[default_flag] == 1]) /len(perc_group_added)) * 100
            except:
                continue

        else:

            perc_group = df[df[credit_score] == i]
            perc_group_added = pd.concat([perc_group_added, perc_group])
            default_rate = (len(perc_group_added[perc_group_added[default_flag] == 1]) /len(df)) * 100

        default_rates.append(default_rate)

    for index,value in enumerate(default_rates):
        if value > target: 
            if index != 0:
                cut_off = index-1
                rate = round(default_rates[cut_off],2)
                break
                
    return print('The optimal cut-off credit score is ' + str(cut_off) + ' and above, to keep below 2.5% default rate, giving a ' + str(rate) + '% default rate')


In [117]:
find_cutoff(df2, 'calibrated_prob percentile_round', 'default_flag', 2.5)

The optimal cut-off credit score is 89 and above, to keep below 2.5% default rate, giving a 2.42% default rate


In [118]:
proba_bins = pd.DataFrame(df2.groupby(['perc_bins'])['orig_prob','calibrated_prob'].mean())

C:\Users\User\AppData\Local\Temp/ipykernel_8244/3771005398.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  proba_bins = pd.DataFrame(df2.groupby(['perc_bins'])['orig_prob','calibrated_prob'].mean())


In [119]:
summary_df2 = summary_df.merge(proba_bins, on = 'perc_bins', how = 'left' )

# Coding Assignment Qn 3: Creating Confidence Intervals

Please create deciles by credit score and provide risk and default levels by deciles (by decile and cumulative). Bonus if you can provide confidence (or methodology how you would do it) for your scores/default rates by bin. 

In [120]:
def create_population(df): 
    row_list = []
    for index,row in df.iterrows(): 
        inner_list = []
        inner_list.extend([0] * row[0])
        inner_list.extend([1] * row[1])
        row_list.append(inner_list)
        
    return row_list

In [121]:
summary_df2['popn'] = create_population(summary_df2)

In [140]:
import random

def sampling(popn):
    inner_list = []
    dist_list = []
    for i in range(0,1000):
        sampled_list = random.sample(popn, 100)
        dist = round(sum(sampled_list)/len(sampled_list), 2)
        
        dist_list.append(dist)
    
    sampled_list_sorted = sorted(dist_list)
    
    perc_2_75 = np.percentile(sampled_list_sorted, 2.75)
    perc_97_5 = np.percentile(sampled_list_sorted, 97.5)
    
    return (perc_2_75, perc_97_5)
        
        

In [141]:
summary_df2['confi_int'] = summary_df2['popn'].apply(lambda x: sampling(x))

In [142]:
summary_df2['confi_int']

0      (0.0, 0.0)
1      (0.0, 0.0)
2      (0.0, 0.0)
3     (0.0, 0.02)
4     (0.0, 0.02)
5     (0.0, 0.03)
6     (0.0, 0.04)
7     (0.0, 0.05)
8    (0.05, 0.16)
9    (0.67, 0.84)
Name: confi_int, dtype: object

In [132]:
summary_df3 = summary_df2.drop(columns = ['popn', 'index'])

In [133]:
summary_df3

,perc_bins,0,1,total_cases,cumulative_neg_case,cumulative_pos_case,default_rate,cumulative_default_rate,orig_prob,calibrated_prob,confi_int
0,"(0, 10]",1573,0,1573,1573,0,0.000000,0.000000,0.000235,0.000289,"(0.0, 0.0)"
1,"(10, 20]",1569,0,1569,3142,0,0.000000,0.000000,0.001028,0.004087,"(0.0, 0.0)"
2,"(20, 30]",1548,0,1548,4690,0,0.000000,0.000000,0.003267,0.012058,"(0.0, 0.0)"
3,"(30, 40]",1561,7,1568,6251,7,0.004464,0.001119,0.009240,0.025818,"(0.0, 0.02)"
4,"(40, 50]",1563,6,1569,7814,13,0.003824,0.001661,0.022544,0.043957,"(0.0, 0.02)"
5,"(50, 60]",1551,10,1561,9365,23,0.006406,0.002450,0.057834,0.068233,"(0.0, 0.02)"
6,"(60, 70]",1541,23,1564,10906,46,0.014706,0.004200,0.128793,0.105230,"(0.0, 0.04)"
7,"(70, 80]",1529,35,1564,12435,81,0.022379,0.006472,0.276617,0.176868,"(0.0, 0.05)"
8,"(80, 90]",1406,158,1564,13841,239,0.101023,0.016974,0.583187,0.296821,"(0.05, 0.16)"
9,"(90, 100]",388,1177,1565,14229,1416,0.752077,0.090508,0.897921,0.569111,"(0.67, 0.83)"
